In [29]:
import pandas as pd

# Load the dataset
df2 = pd.read_csv("/content/drive/MyDrive/DL Project/pokemon_data.csv")
df = pd.read_csv("/content/drive/MyDrive/DL Project/pokemon.csv", index_col=0)
# Display the first few rows of the dataset
df.head()


,height,weight,hp,attack,defense,sp_atk,sp_def,speed,total,catch_rate,...,egg_cycles_5,egg_cycles_50,egg_cycles_80,special_group_Baby Pokemon,special_group_Fossil,special_group_Future Paradox,special_group_Legendary,special_group_Mythical,special_group_Ordinary,special_group_Ultra Beast
0,0.166667,0.022025,0.131579,0.19,0.155556,0.368421,0.382022,0.222222,0.262385,0.166667,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1,0.291667,0.045595,0.328947,0.32,0.311111,0.526316,0.550562,0.388889,0.422018,0.166667,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
2,0.708333,0.381762,0.592105,0.52,0.533333,0.736842,0.775281,0.611111,0.642202,0.166667,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
3,0.125000,0.028207,0.052632,0.22,0.088889,0.315789,0.213483,0.444444,0.245872,0.166667,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
4,0.333333,0.068779,0.302632,0.34,0.255556,0.526316,0.382022,0.611111,0.422018,0.166667,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0


In [30]:
# Select features
features1 = df

# The target variable
target = df2['generation']


In [8]:
pip install torch_geometric

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 8.0 MB/s eta 0:00:00


In [31]:
from sklearn.preprocessing import StandardScaler
# Check correlations with the dependent variable
correlations = features1.corrwith(target).abs().sort_values(ascending=False)
print(correlations.head(35))  # Print top 35 correlations
# Select top features based on correlation
top_features = correlations.head(1554).index.tolist()
node_features1 = features1[top_features]
scaler = StandardScaler()
node_features = scaler.fit_transform(node_features1)

egg_cycles_50                       0.249244
species_Paradox Pokémon             0.211912
ability2_No ability2                0.200865
base_exp_285                        0.200732
genderless                          0.143627
hidden_ability_No hidden ability    0.143042
egg_group1_Undiscovered             0.141823
type2_Normal                        0.139907
ability1_Protosynthesis             0.139535
special_group_Future Paradox        0.139535
ability1_Quark Drive                0.139535
base_exp_265                        0.131114
attack                              0.128675
type2_Ghost                         0.127661
type2_Flying                        0.123993
ability1_Rock Head                  0.122257
percent_male                        0.121922
total                               0.121852
hp                                  0.117853
ev_yield_3 Attack                   0.116418
base_friendship_35                  0.116345
egg_group2_Mineral                  0.115815
special_gr

In [13]:
import torch

!pip uninstall torch-scatter torch-sparse torch-geometric torch-cluster  --y
!pip install torch-scatter -f https://data.pyg.org/whl/torch-{torch.__version__}.html
!pip install torch-sparse -f https://data.pyg.org/whl/torch-{torch.__version__}.html
!pip install torch-cluster -f https://data.pyg.org/whl/torch-{torch.__version__}.html
!pip install git+https://github.com/pyg-team/pytorch_geometric.git

Found existing installation: torch_geometric 2.5.3
Uninstalling torch_geometric-2.5.3:
  Successfully uninstalled torch_geometric-2.5.3
Looking in links: https://data.pyg.org/whl/torch-2.3.0+cu121.html
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.9/10.9 MB 74.2 MB/s eta 0:00:00
Looking in links: https://data.pyg.org/whl/torch-2.3.0+cu121.html
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.1/5.1 MB 21.2 MB/s eta 0:00:00
Looking in links: https://data.pyg.org/whl/torch-2.3.0+cu121.html
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 39.4 MB/s eta 0:00:00
  Cloning https://github.com/pyg-team/pytorch_geometric.git to /tmp/pip-req-build-ah3n7eid
  Running command git clone --filter=blob:none --quiet https://github.com/pyg-team/pytorch_geometric.git /tmp/pip-req-build-ah3n7eid
  Resolved https://github.com/pyg-team/pytorch_geometric.git to commit fbafbc4fc9181e8759ec1f39d9618992793b5fe1
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  P

In [32]:
from sklearn.neighbors import kneighbors_graph

# Create a k-nearest neighbors graph
k = 5  # Number of neighbors
adjacency_matrix = kneighbors_graph(node_features, k, mode='connectivity', include_self=False)

# Convert the adjacency matrix to edge indices
import torch
from torch_geometric.utils import from_scipy_sparse_matrix

edge_index = from_scipy_sparse_matrix(adjacency_matrix)[0]


In [33]:
import torch_geometric
from torch_geometric.data import Data
from torch_geometric.typing import SparseTensor

# Create PyG Data object
num_nodes = node_features.shape[0]
adj = SparseTensor(
    row=edge_index[0].to(torch.int64),
    col=edge_index[1].to(torch.int64),
    sparse_sizes=(num_nodes, num_nodes),
    is_sorted=True,
    trust_data=True,
)

adj_t = adj.to_symmetric()

pyg_data = Data(
    x=torch.tensor(node_features, dtype=torch.float),
    edge_index=edge_index,
    adj_t=adj_t,
    y=torch.tensor(target.values, dtype=torch.long).unsqueeze(1),
)

# Split indices (assuming a 80-10-10 split)
from sklearn.model_selection import train_test_split

train_idx, test_idx = train_test_split(range(num_nodes), test_size=0.1, random_state=42)
train_idx, val_idx = train_test_split(train_idx, test_size=0.11, random_state=42)

split_idx = {
    'train': torch.tensor(train_idx),
    'valid': torch.tensor(val_idx),
    'test': torch.tensor(test_idx),
}
pyg_data.split_idx = split_idx
pyg_data.num_classes = len(target.unique())


In [34]:
import torch.nn as nn
import torch.nn.functional as F
from torch_geometric.nn import GCNConv

class GCN(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim):
        super(GCN, self).__init__()
        self.conv1 = GCNConv(input_dim, hidden_dim)
        self.conv2 = GCNConv(hidden_dim, hidden_dim)
        self.conv3 = GCNConv(hidden_dim, output_dim)

    def forward(self, data):
        x, edge_index = data.x, data.edge_index
        x = self.conv1(x, edge_index)
        x = F.relu(x)
        x = self.conv2(x, edge_index)
        x = F.relu(x)
        x = self.conv3(x, edge_index)
        return F.log_softmax(x, dim=1)

# Initialize the model, optimizer, and loss function
num_classes = int(pyg_data.y.max())+1  # Calculate number of classes from data
model = GCN(input_dim=node_features.shape[1], hidden_dim=128, output_dim=num_classes)
optimizer = torch.optim.SGD(model.parameters(), lr=0.04, momentum=0.85)
criterion = nn.CrossEntropyLoss()

In [35]:
def train(model, data, optimizer, criterion):
    model.train()
    optimizer.zero_grad()
    out = model(data)
    loss = criterion(out[data.split_idx['train']], data.y[data.split_idx['train']].squeeze())
    loss.backward()
    optimizer.step()
    return loss.item()

def evaluate(model, data):
    model.eval()
    out = model(data)
    _, pred = out.max(dim=1)

    correct = (pred[data.split_idx['test']] == data.y[data.split_idx['test']].squeeze()).sum()
    acc = correct / data.split_idx['test'].size(0)
    return acc.item()


In [36]:
# Training loop
from sklearn.model_selection import train_test_split
from random import randrange

# for i in range(10):
# seed=randrange(40,100)
seed=randrange(40,100)
# Split indices (assuming a 80-10-10 split)

train_idx, test_idx = train_test_split(range(num_nodes), test_size=0.1, random_state=seed)
train_idx, val_idx = train_test_split(train_idx, test_size=0.11, random_state=seed)

split_idx = {
    'train': torch.tensor(train_idx),
    'valid': torch.tensor(val_idx),
    'test': torch.tensor(test_idx),
}
pyg_data.split_idx = split_idx
pyg_data.num_classes = len(target.unique())
epochs = 1000
for epoch in range(epochs):

  loss = train(model, pyg_data, optimizer, criterion)
  acc = evaluate(model, pyg_data)

  print(f'Epoch {epoch+1}, Train Loss: {loss:.4f}, Test Accuracy: {acc:.4f}')


Epoch 1, Train Loss: 2.8148, Test Accuracy: 0.1683
Epoch 2, Train Loss: 2.5195, Test Accuracy: 0.1980
Epoch 3, Train Loss: 2.3490, Test Accuracy: 0.1782
Epoch 4, Train Loss: 2.1988, Test Accuracy: 0.1584
Epoch 5, Train Loss: 2.1100, Test Accuracy: 0.1782
Epoch 6, Train Loss: 1.9629, Test Accuracy: 0.2574
Epoch 7, Train Loss: 1.7941, Test Accuracy: 0.2970
Epoch 8, Train Loss: 1.6329, Test Accuracy: 0.3267
Epoch 9, Train Loss: 1.4672, Test Accuracy: 0.3366
Epoch 10, Train Loss: 1.3615, Test Accuracy: 0.3564
Epoch 11, Train Loss: 1.2571, Test Accuracy: 0.3564
Epoch 12, Train Loss: 1.1337, Test Accuracy: 0.3861
Epoch 13, Train Loss: 1.0262, Test Accuracy: 0.3960
Epoch 14, Train Loss: 0.9324, Test Accuracy: 0.3960
Epoch 15, Train Loss: 0.8483, Test Accuracy: 0.4158
Epoch 16, Train Loss: 0.7692, Test Accuracy: 0.4653
Epoch 17, Train Loss: 0.6899, Test Accuracy: 0.4455
Epoch 18, Train Loss: 0.6219, Test Accuracy: 0.4356
Epoch 19, Train Loss: 0.5571, Test Accuracy: 0.4455
Epoch 20, Train Loss: